## Obtaining results

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
while 'notebooks' in os.getcwd():
    os.chdir('..')

import numpy as np
from src.simulations import PathSimulator
from src.gpu.simulation_based_tree import SimulationBasedModel
import torch
from time import time

In [3]:
kappa = 5.0
K = 10
r = 0.1
eta = 0.9
theta = 0.16
rho = 0.1
T = 0.25
v0 = 0.25**2
# S0 = 10
M = 10_000

In [4]:
simulator = PathSimulator()

In [5]:
[3] + [1,2,3]

[3, 1, 2, 3]

In [6]:
reference_values = {
    8: 2,
    9: 1.1076,
    10: 0.52,
    11: 0.2137,
    12: 0.0820

}

In [14]:
for n in [100, 200, 500]:
    m = 1000*n

    print(f"\n-------------------- N = {n} -------------------------\n")

    for S0 in [8,9,10,11,12]:

        dt = T/n

        t0 = time()
        # Mean and covariance for the multivariate normal distribution
        mean = torch.zeros(2, device='cuda')
        cov = torch.tensor([[1, rho], [rho, 1]], device='cuda')

        # Multivariate normal distribution
        mvn = torch.distributions.MultivariateNormal(mean, cov)

        # Generate samples (size: m x n x 2)
        brownian = mvn.sample((m, n)) * torch.sqrt(torch.tensor(dt, device='cuda'))

        # Extract dW and dW_hat
        dW = brownian[:, :, 0]
        dW_hat = brownian[:, :, 1]

        St, vt = simulator.heston_gpu(
            T,
            m,
            n,
            rho,
            kappa,
            theta,
            v0,
            S0,
            eta,
            r
        )
        pricer = SimulationBasedModel(n, K, r, T/n, option_type='american')

        option_price_proba = pricer.compute_option_prices_probabilities(
            St,
            vt,
            lambda x,k: torch.maximum(k-x,torch.tensor(0)),
            n,
            T,
            K,
            r,
            v0
        )
        option_price = pricer.compute_option_prices_counting(St,  lambda x,k: torch.maximum(k-x,torch.tensor(0)),)

        t1 = time()

        print(f"S0 = {S0}. Reference value: {reference_values[S0]}. Counting approach: {option_price.item()} . Probabilities approach: {option_price_proba}. ({round(t1-t0, 2)} s) ")


-------------------- N = 100 -------------------------



/users/eleves-a/2022/pedro.silva/Desktop/EA_recherche/src/gpu/simulation_based_tree.py:181: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cdf_values = self.torch_norm_cdf(x)


S0 = 8. Reference value: 2. Counting approach: 2.0 . Probabilities approach: 2.0876898765563965. (0.32 s) 
S0 = 9. Reference value: 1.1076. Counting approach: 1.0827258825302124 . Probabilities approach: 1.1691699028015137. (0.3 s) 
S0 = 10. Reference value: 0.52. Counting approach: 0.4963997006416321 . Probabilities approach: 0.5206362009048462. (0.3 s) 
S0 = 11. Reference value: 0.2137. Counting approach: 0.19463574886322021 . Probabilities approach: 0.18817515671253204. (0.3 s) 
S0 = 12. Reference value: 0.082. Counting approach: 0.07085494697093964 . Probabilities approach: 0.05675114691257477. (0.3 s) 

-------------------- N = 200 -------------------------

S0 = 8. Reference value: 2. Counting approach: 2.0 . Probabilities approach: 2.066905975341797. (0.73 s) 
S0 = 9. Reference value: 1.1076. Counting approach: 1.093392014503479 . Probabilities approach: 1.1524524688720703. (0.73 s) 
S0 = 10. Reference value: 0.52. Counting approach: 0.5120230913162231 . Probabilities approach: 

In [28]:
for n in [100, 200, 500]:

    print(f"\n-------------------- N = {n} -------------------------\n")

    for S0 in [8,9,10,11,12]:

        St, vt = simulator.heston(
            T,
            n*1000,
            n,
            rho,
            kappa,
            theta,
            v0,
            S0,
            eta,
            r
        )

        g = lambda x, k : np.maximum(k-x, 0) # put option

        t0 = time()
        option_price_counting = pricer.compute_option_prices_counting(St, vt, g, n, T, K, r, v0)
        t1 = time()
        option_price_probabilities = pricer.compute_option_prices_probabilities(St, vt, g, n, T, K, r,v0)
        t2 = time()

        print(f"S0 = {S0}. Reference value: {reference_values[S0]}. Counting approach: {round(option_price_counting, 4)} ({round(t1-t0, 2)} s). Probabilities approach: {round(option_price_probabilities, 4)} ({round(t2-t1, 2)} s)")


-------------------- N = 100 -------------------------

S0 = 8. Reference value: 2. Counting approach: 2.0 (5.95 s). Probabilities approach: 2.0814 (4.47 s)
S0 = 9. Reference value: 1.1076. Counting approach: 1.0911 (5.95 s). Probabilities approach: 1.171 (4.48 s)
S0 = 10. Reference value: 0.52. Counting approach: 0.5066 (6.0 s). Probabilities approach: 0.5377 (4.45 s)
S0 = 11. Reference value: 0.2137. Counting approach: 0.2006 (5.94 s). Probabilities approach: 0.203 (4.48 s)
S0 = 12. Reference value: 0.082. Counting approach: 0.0712 (5.92 s). Probabilities approach: 0.0611 (4.41 s)

-------------------- N = 200 -------------------------

S0 = 8. Reference value: 2. Counting approach: 2.0 (24.0 s). Probabilities approach: 2.062 (20.27 s)
S0 = 9. Reference value: 1.1076. Counting approach: 1.1 (24.55 s). Probabilities approach: 1.1531 (19.88 s)
S0 = 10. Reference value: 0.52. Counting approach: 0.5144 (24.5 s). Probabilities approach: 0.5368 (19.86 s)
S0 = 11. Reference value: 0.2137. 